In [1]:
# Se importan la librerías necesarias
import customtkinter as ctk
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
Crit = 2 # Max difference accepted # defined by... company?

In [2]:
# Función para comparar dos colores
def deltaE2000  (L,a,b,prt):
  hp = np.zeros(2) # Se inicia el vector
  Lm = (L[0]+L[1])/2 #L promedio
  C = (a**2+b**2)**(1/2)
  #print("C: ",C)
  Cm = (C[0]+C[1])/2 #C promedio
  G = (1/2)*(1-(Cm**(7)/(Cm**(7)+25**7))**(1/2))
  ap = a*(1+G) # Se redefine a
  Cp = (ap**2+b**2)**(1/2) # Se calcula C con la nueva a
  Cmp = (Cp[0]+Cp[1])/2 #Cp promedio
  for i in range (2):
    if (np.degrees(np.arctan2(b[i],ap[i]))>=0):
      hp[i] = np.degrees(np.arctan2(b[i],ap[i]))
    else:
      hp[i] = np.degrees(np.arctan2(b[i],ap[i])) + 360
      #print("else1")
  if (abs(hp[0]-hp[1])>180):
    Hmp = (hp[0]+hp[1])/2+180
    #print("else2")
  else:
    Hmp = (hp[0]+hp[1])/2
  T = 1 - 0.17*1*(np.cos(np.radians(Hmp-30))) \
    + 0.24*1*(np.cos(np.radians(2*Hmp))) \
    + 0.32*1*(np.cos(np.radians(3*Hmp+6))) \
    - 0.2*1*(np.cos(np.radians(4*Hmp-63)))
  if (abs(hp[1]-hp[0])<=180):
    dhp = hp[1]-hp[0]
  elif ((abs(hp[1]-hp[0])>180) and hp[1]<=hp[0]):
    dhp = hp[1]-hp[0] + 360
    #print("else3")
  else:
    dhp = hp[1]-hp[0] - 360
    #print("else4")
  dLp = L[1]-L[0]
  dCp = Cp[1]-Cp[0]
  dHp = 2*((Cp[0]*Cp[1])**(1/2))*1*(np.sin(np.radians(dhp/2)))
  SL = 1+(0.015*(Lm-50)**2)/((20+(Lm-50)**2)**(1/2))
  SC = 1+0.045*Cmp
  SH = 1+0.015*Cmp*T
  dtetha=30*np.exp(-((Hmp-275)/25)**2)
  RC = 2*((Cmp**7)/(Cmp**(7)+25**7))**(1/2)
  RT = -RC*1*(np.sin(np.radians(2*dtetha)))
  KL=KC=KH=1 # Por defecto
  DE00 = ((dLp/(KL*SL))**2+(dCp/(KC*SC))**2+(dHp/(KH*SH))**2 \
          +RT*(dCp/(KC*SC))*(dHp/(KH*SH)))**(1/2) # Actualizar notación
  if (prt == 1):
    print("DE00 = ",DE00)
  elif (prt == 2):
    print("DE00 = ",DE00)
    print("DL = ","%.2f" % (L[0]-L[1]),", Da = ","%.2f" %(a[0]-a[1]),", Db = ",
          "%.2f" %(b[0]-b[1]))
  return DE00

In [3]:
# Función para hacer arrays con 2 elementos iguales
def count (L,a,b):
  Lrr = np.array([float(L),float(L)])
  arr = np.array([float(a),float(a)])
  brr = np.array([float(b),float(b)])
  k = 0 # se reinicia el valor de k
  return (Lrr, arr, brr, k)

In [4]:
# Función para encontrar los límites de los delta L, a y b
def Delta_Limits (L0,a0,b0,aux):
  import time # for testing times
  MaxDE = 2.0 # Se establece el valor máximo del delta E aceptado
  Precision = 0.001 # Se establece la precisión
  Maxk = 1000 # Máximo de iteraciones por ciclo
  dmas = 0.5 # Cantidad que se suma en cada ciclo mientras el delta sea menor
  dmenos = 0.002 #Cantidad que se resta en los ciclo mientras el delta sea mayor
  t0 = time.time() # Se toma el tiempo inicial

  # delta L
  L,a,b,k = count (L0,a0,b0)
  DE00 = deltaE2000(L,a,b,0) #should be zero
  # Mientras la diferencia sea mayor a la precisión deseada, y no se exceda k
  while (abs(DE00-MaxDE)>Precision and k<Maxk):
    if (DE00 < MaxDE):
      L[1] = L[1] + dmas
    elif (DE00 > MaxDE):
      L[1] -= dmenos
    DE00 = deltaE2000(L,a,b,0)
    k = k+1
  DL = abs(L[1]-L[0])
  kL = k

  # delta a
  L,a,b,k = count (L0,a0,b0) #reset values
  DE00 = deltaE2000(L,a,b,0)
  while (abs(DE00-MaxDE)>Precision and k<Maxk):
    if (DE00 < MaxDE):
      a[1] = a[1] + dmas
    elif (DE00 > MaxDE):
      a[1] -= dmenos
    DE00 = deltaE2000(L,a,b,0)
    k = k+1
  Da = abs(a[1]-a[0])
  ka = k

  # delta b
  L,a,b,k = count (L0,a0,b0) #reset values
  DE00 = deltaE2000(L,a,b,0)
  while (abs(DE00-MaxDE)>Precision and k<Maxk):
    if (DE00 < MaxDE):
      b[1] = b[1] + dmas
    elif (DE00 > MaxDE):
      b[1] -= dmenos
    DE00 = deltaE2000(L,a,b,0)
    k = k+1
  Db = abs(b[1]-b[0])
  kb = k
  t1 = time.time() # Se toma el tiempo final
  if (aux == 2):
    #print("k:",kL,ka,kb)
    print("Max DL:",round(DL,3)," Da:",round(Da,3)," Db:",round(Db,3))
    print("Iteraciones totales: ", kL+ka+kb)
    print("Time: ",round(t1-t0,4),"s")
  return DL,Da,Db

In [5]:
def input_vector (L,a,b): # transforms a vector of deltas to a vector of values
  lenght = len(L)
  # The function only works if the lenght of the three vectors is equal
  if (len(L) == len(a) & len(a) == len(b)):
    Lt = np.zeros(lenght) # initializing vectors
    at = np.zeros(lenght)
    bt = np.zeros(lenght)
    Lt[0] = float(L[0])
    at[0] = float(a[0])
    bt[0] = float(b[0])
    for i in range (lenght-1): # -1 to coincide with next comment
      Lt[i+1] = float(L[0]+L[i+1]) # +1 to avoid adding into the first value
      at[i+1] = float(a[0]+a[i+1])
      bt[i+1] = float(b[0]+b[i+1])
    return Lt,at,bt
  else:
    print("the vectors must have the same lenght")

In [6]:
def elipse(center_x,center_y,width,height):
  # Generate points for the ellipse
  t = np.linspace(0, 2 * np.pi, 360)
  x = center_x + width * np.cos(t)
  y = center_y + height * np.sin(t)
  # Plot the ellipse
  plt.plot(x, y,color="red")

In [7]:
def rectangle(center_x,center_y,width,height,ax,color_):
  # we draw a rectangle
  rectangleCielab=plt.Rectangle(xy=(center_x-width/2,center_y-height/2),
          width=width, height=height, color=color_, fill=False, linewidth=3)
  # add the patch to the Axes
  ax.add_patch(rectangleCielab)

In [8]:
def plot_config(center_x,center_y,x_axis,y_axis,graph_type):
  # Set axis limits
  ax = plt.gca()

  # Plot displaced y axis
  plt.axhline(y=center_y,color='k', linestyle = (0,(5,10)),linewidth=0.6)

  # We verify which limit is bigger
  if (x_axis>y_axis):
    axis = 2*x_axis
    n = x_axis//y_axis + 1 # plus 1 because it doesn't include the edges
    m = 2 + 1
    if n < 3:
      n = 3
  else:
    axis = 2*y_axis
    m = y_axis//x_axis + 1
    n = 2 + 1
    if m < 3:
      m = 3
  
  print("n y m:",n,m)
  if (graph_type == "L"):
    # Label
    plt.xlabel('ΔL')

    # set the figure limits for L
    x_downlimit,x_uplimit = (-x_axis, x_axis)
    y_downlimit,y_uplimit = (center_y-0.7*axis, center_y+0.7*axis)
    ax.set_xlim([x_downlimit,x_uplimit])
    ax.set_ylim([y_downlimit,y_uplimit])
    print("x, y ticks",x_axis,y_axis)

  elif (graph_type == "C"):
    # Labels
    plt.xlabel('Δa')
    plt.ylabel('Δb')

    # Plot displaced x axis
    plt.axvline(x=center_x,color='k', linestyle = (0,(5,10)),linewidth=0.6)

    # set the figure limits
    x_downlimit,x_uplimit = (center_x-axis,center_x+axis)
    y_downlimit,y_uplimit = (center_y-axis,center_y+axis)
    ax.set_xlim([x_downlimit,x_uplimit])
    ax.set_ylim([y_downlimit,y_uplimit])

  else:
    print("Invalid graph type \nRestart the session xd")
    #%matplotlib agg # to avoid graph from showing
    return

  ax.set_aspect('equal') # both axis with same scaling

  """# List of ticks
  x_ticks_vector = np.arange(-n,n+0.1,1) # from -n to n, with pace of 1
  y_ticks_vector = np.arange(-m,m+0.1,1) # +0.1 beacause last element is cut

  # The vector (of integers) multiplied by the axis to scale it plus the center
  # for the offset. The 2/n allows the tiks remain inside the axis limits
  x_ticks = center_x + x_ticks_vector*x_axis*2/n
  y_ticks = center_y + y_ticks_vector*y_axis*2/m"""

  # List of ticks
  x_ticks_vector = np.arange(x_downlimit,x_uplimit+0.1,axis/n) # from -n to n, with pace of 1
  y_ticks_vector = np.arange(y_downlimit,y_uplimit+0.1,axis/m) # +0.1 beacause last element is cut
  print(axis/m)
  # The vector (of integers) multiplied by the axis to scale it plus the center
  x_ticks = x_ticks_vector
  y_ticks = y_ticks_vector


  # Applays the list of y_ticks
  ax.set_yticks(y_ticks)

  # Display 1 decimal in y_ticks
  ax.set_yticklabels([f"{label:.1f}" for label in y_ticks])

  if (graph_type == "L"):
    ax.set_xticks([]) # hide ticks in X axis (they don't have physical meaning)

  elif (graph_type == "C"):
    # Applays the list of x_ticks
    ax.set_xticks(x_ticks)

    # Display 1 decimal in x_ticks
    ax.set_xticklabels([f"{label:.1f}" for label in x_ticks])

  return ax # necesary for other plots in the figure

In [9]:
# Función para graficar el plano cromático (a,b)
def graph (L,a,b,aux,aux2):
  # aux defines how much to print in deltaE2000 (0, 1 or 2)
  # aux2 chooses between ploting only DE00 or adding Cielab (0 or 1)
  DL,Da,Db = Delta_Limits(L[0],a[0],b[0],aux)
  # Define the center, major axis, and minor axis
  center_x = a[0]
  center_y = b[0]
  x_axis = width =  Da
  y_axis = height =  Db

  ax = plot_config(center_x,center_y,x_axis,y_axis,"C")

  elipse(center_x,center_y,width,height) #function to graph the elipse

  # Add title
  plt.title('Chromatic plane')

  # Plot the colors
  plt.plot(center_x,center_y,'ko')
  plt.plot(a[1],b[1],'ro')

  if (aux2 == 1): #comparative rectangle
    DL=Da=Db = Crit
    # Define the center, major axis, and minor axis
    center_x = a[0]
    center_y = b[0]
    x_axis = width =  Da
    y_axis = height =  Db
    # we draw a rectangle
    rectangle(center_x,center_y,width*2,height*2,ax,"red") # plots the rectangle

  # saving the plot
  plt.savefig('Chromatic_plot2.png',bbox_inches="tight") 

  # show the figure
  plt.show()

  # close the figure
  plt.close()

  # Show DE00
  deltaE2000(L,a,b,aux)

In [10]:
def graphL (L,a,b,aux,aux2):
  DL,Da,Db = Delta_Limits(L[0],a[0],b[0],aux)
  # Define the center and the lenght
  center = L[0]
  height = 2*DL # for DE00
  heightLab = Crit*2 # for Cielab
  width = height/1.5 # for aestetics
  if (aux == -1):
    print("Max DL: ", DL)

  # Initiate the graph
  plt.figure()
  # Plot the colors
  plt.plot(0,center,'ko')
  plt.plot(0,L[1],'ro')

  ax = plot_config(0,center,width,height,"L")

  if (aux2==1): # comparative plot
    # we draw a rectangle (Cielab)
    rectangle(0,center,width,heightLab,ax,"blue") # plots the Lab rectangle

  rectangle(0,center,width,height,ax,"red") # plots the rectangle

  # Add title
  plt.title('Lightness plane')

  # saving the plot
  plt.savefig('Light_plot2.png',bbox_inches="tight") 

  # show the figure
  plt.show()

  # close the figure
  plt.close()

In [11]:
# Function to graph in Cielab
def graph_Cielab (L,a,b,aux):
  DL,Da,Db = (L[0]-L[1],a[0]-a[1],b[0]-b[1]) # Current deltas
  # Define the center, major axis, and minor axis
  center_x = a[0]
  center_y = b[0]
  x_axis = width =  Crit
  y_axis = height =  Crit

  # Initiate the graph
  plt.figure()
  # Plot the colors
  plt.plot(center_x,center_y,'ko')
  plt.plot(a[1],b[1],'ro')

  # get the current figure
  fig = plt.gcf()
  # get the current Axes instance
  ax = fig.gca()

  # set the figure limits # The abs() is to avoid inverted axis
  ax.set_xlim([center_x-2*abs(x_axis), center_x+2*abs(x_axis)])
  ax.set_ylim([center_y-2*abs(y_axis), center_y+2*abs(y_axis)])
  ax.set_aspect('equal') # both axis with same scaling

  # List of ticks
  x_ticks = (center_x-2*width,center_x-width,center_x,center_x+width,center_x+2*width)
  y_ticks = (center_y-2*height,center_y-height,center_y,center_y+height,center_y+2*height)
  # Applays the list of ticks
  ax.set_xticks(x_ticks)
  ax.set_yticks(y_ticks)
  # Display 1 decimal in ticks
  ax.set_xticklabels([f"{label:.1f}" for label in x_ticks])
  ax.set_yticklabels([f"{label:.1f}" for label in y_ticks])

  # we draw a rectangle
  rectangle(center_x,center_y,width*2,height*2,ax,"red") # plots the rectangle

  # Add labels and title
  plt.ylabel('Δb')
  plt.xlabel('Δa')
  plt.title('Chromatic plane (Cielab)')

  # Plot displaced axis
  plt.axhline(y=center_y,color='k', linestyle = (0,(5,10)),linewidth=0.6)
  plt.axvline(x=center_x,color='k', linestyle = (0,(5,10)),linewidth=0.6)

  # show the figure
  plt.show()

  # close the figure
  plt.close()

  # Show DE
  if (aux == 1):
    print("DE = ",(DL**2+Da**2+Db**2)**(1/2))

In [12]:
def graphL_Cielab (L,a,b):
  # Define the center and the lenght
  center = L[0]
  height = Crit*2 # for Cielab
  width = height/1.5 # for aestetics

  # Initiate the graph
  plt.figure()
  # Plot the colors
  plt.plot(0,center,'ko')
  plt.plot(0,L[1],'ro')

  ax = plot_config(0,center,width,height,"L")

  # we draw a rectangle (Cielab)
  rectangle(0,center,width,height,ax,"red") # plots the rectangle

  # Add title
  plt.title('Lightness plane (Cielab)')

  # show the figure
  plt.show()

  # Close the plot
  plt.close()

In [13]:
def total_graph (L,a,b,aux,case_n):
  # case 0: only DE00
  if (case_n == 0):
    graph(L,a,b,aux,0)
    graphL(L,a,b,aux,0)
  # case 1: all graphs in different figures
  if (case_n == 1):
    graph(L,a,b,aux,0)
    graph_Cielab(L,a,b,aux)
    graphL(L,a,b,aux,0)
    graphL_Cielab(L,a,b)
  # case 2: comparative graphs
  if (case_n == 2):
    graph(L,a,b,aux,1)
    graphL(L,a,b,aux,1)

In [14]:
def comparison (L,a,b,aux,pattern):
  # patron says how much data was taken from the pattern
  Lm = am = bm = 0 # we start the variables
  for i in range (pattern):
    Lm += L[i]/pattern # we take the mean of the pattern measures
    am += a[i]/pattern
    bm += b[i]/pattern

  DL,Da,Db = Delta_Limits(Lm,am,bm,aux)
  # Define the center, major axis, and minor axis
  center_x = am
  center_y = bm
  x_axis = width =  Da
  y_axis = height =  Db

  # Initiate the graph
  plt.figure()

  # Add labels and title
  plt.title('Chromatic plane')

  ax = plot_config(center_x,center_y,x_axis,y_axis,"C")

  # Graph the elipse
  elipse(center_x,center_y,width,height)

  # Plot the colors
  for i in range (pattern):
    plt.plot(a[i],b[i],color="purple",marker='o')
    print("i:",i,"a:",a[i],"b:",b[i])
  for i in range (len(a)-pattern):
      plt.plot(a[pattern+i],b[pattern+i],'ro')

  # Show the plot
  plt.show()

  # Close the plot
  plt.close()

  # Show DE00
  deltaE2000(L,a,b,aux)

In [15]:
# Register of values measured
def Register (L,a,b,Reg):
    Ltext = open("L_text","a+") # opens a .txt
    Ltext.write(str(L)+"\n") # writes the value in the txt
    atext = open("a_text","a+")
    atext.write(str(a)+"\n")
    btext = open("b_text","a+")
    btext.write(str(b)+"\n")
    Names = open("Nombre","a+") 

    Names.write(str(Reg)+"\n") # Register the values of the pattern with their name

In [16]:
# GUI
class App(ctk.CTk):
    
    def __init__(self):
        super().__init__()
        
        screen_width = self.winfo_screenwidth() # get current screen width and height
        screen_height = self.winfo_screenheight()
        img_width= screen_width/3 # Sets images width 
        img_height= screen_width/3 # Sets images height 

        self.title("Color Difference")
        #self.geometry(f"{screen_width}x{screen_height}")
        #self.geometry("900x600")
        self.geometry(f"{screen_width}x{screen_height}")

      

        # set grid layout 
        self.grid_columnconfigure(3, weight=5)
        self.grid_columnconfigure(4, weight=3)

        # Load images       
        #self.fox_image = ctk.CTkImage(Image.open('Fox.jpg'))
        self.Chromatic_image0 = ctk.CTkImage(Image.open('Blank.png'),size=(img_width,img_height))
        self.Light_image0 = ctk.CTkImage(Image.open('Blank.png'),size=(img_width/2,img_height))

        # labels
        self.Title_label = ctk.CTkLabel(self,text="Pruebas de Color",font=ctk.CTkFont(size=30, weight="bold"))
        self.Title_label.grid(row=0, column=0, pady=10,columnspan=5)

        self.Data_label = ctk.CTkLabel(self,text="Datos",font=ctk.CTkFont(size=20))
        self.Data_label.grid(row=1, column=1, pady=10,columnspan=2)

        self.Data_label = ctk.CTkLabel(self,text="Gráficas",font=ctk.CTkFont(size=20))
        self.Data_label.grid(row=1, column=3, pady=10,columnspan=2)

        self.Result_label = ctk.CTkLabel(self,text="Resultado",font=ctk.CTkFont(size=20),fg_color=("gray50","gray50"),
                                         corner_radius=0)
        self.Result_label.grid(row=7, column=1, pady=10,sticky="nsew",columnspan=2)

        self.Patron_label = ctk.CTkLabel(self,text="Patrón:",font=ctk.CTkFont(size=18))
        self.Patron_label.grid(row=2, column=1)#,padx=10)

        self.Diferencia_label = ctk.CTkLabel(self,text="Diferencia:",font=ctk.CTkFont(size=18)) 
        self.Diferencia_label.grid(row=2, column=2)#,padx=10)

        self.L_label = ctk.CTkLabel(self,text="L:") # ,font=ctk.CTkFont(size=20)
        self.L_label.grid(row=3, column=0)#,padx=10)

        self.a_label = ctk.CTkLabel(self,text="a:") 
        self.a_label.grid(row=4, column=0)

        self.b_label = ctk.CTkLabel(self,text="b:") 
        self.b_label.grid(row=5, column=0)

        self.DE_label = ctk.CTkLabel(self,text="ΔE:") 
        self.DE_label.grid(row=7, column=0,padx=20)


        # entries
        entry_conf = {'corner_radius':5, 'fg_color':("gray10","gray90"),# standard entry configuration
                    'text_color':("black")}
        
        self.entryL = ctk.CTkEntry(self,placeholder_text="Inserte un número",**entry_conf)
        self.entryL.grid(row=3, column=1)

        self.entrya = ctk.CTkEntry(self,placeholder_text="Inserte un número",**entry_conf)
        self.entrya.grid(row=4, column=1)

        self.entryb = ctk.CTkEntry(self,placeholder_text="Inserte un número",**entry_conf)
        self.entryb.grid(row=5, column=1)

        self.entrydL = ctk.CTkEntry(self,placeholder_text="Inserte la diferencia",**entry_conf)
        self.entrydL.grid(row=3, column=2)

        self.entryda = ctk.CTkEntry(self,placeholder_text="Inserte la diferencia",**entry_conf)
        self.entryda.grid(row=4, column=2)

        self.entrydb = ctk.CTkEntry(self,placeholder_text="Inserte la diferencia",**entry_conf)
        self.entrydb.grid(row=5, column=2)

        self.entryRegister = ctk.CTkEntry(self,placeholder_text="Nombre el registro",**entry_conf)
        self.entryRegister.grid(row=11, column=2)

        # Buttons
        # "lambda:" allows to call the function with arguments (normally it isn't possible for bottons)
        self.calcular_button = ctk.CTkButton(self,corner_radius=4,text="Calcular",fg_color=("#1f6aa5"),
                                          command=lambda:self.Calc_DE00(img_width,img_height),font=ctk.CTkFont(size=20))
        self.calcular_button.grid(row=6, column=1, pady=10, sticky="ns",columnspan=2)

        self.siguiente_button = ctk.CTkButton(self,corner_radius=4,text="Siguiente Medida",fg_color=("#1f6aa5"),
                                          command=self.Next,font=ctk.CTkFont(size=20))
        self.siguiente_button.grid(row=8, column=1, pady=10, sticky="ns",columnspan=2)
        
        self.Limpiar_button = ctk.CTkButton(self,corner_radius=4,text="Limpiar",fg_color=("#1f6aa5"),
                                          command=self.Limpiar,font=ctk.CTkFont(size=20))
        self.Limpiar_button.grid(row=10, column=3, pady=10, sticky="ns")

        self.registrar_button = ctk.CTkButton(self,corner_radius=4,text="Guardar",fg_color=("#1f6aa5"),
                                          command=self.Registrar,font=ctk.CTkFont(size=20))
        self.registrar_button.grid(row=10, column=4, pady=10, sticky="ns")

        # Images
        self.Chromatic_image_label = ctk.CTkLabel(self,text="",image=self.Chromatic_image0,compound="left")
        self.Chromatic_image_label.grid(row=2, column=3, pady=10, sticky="nsew",rowspan = 8)

        self.Light_image_label = ctk.CTkLabel(self,text="",image=self.Light_image0,compound="left")
        self.Light_image_label.grid(row=2, column=4, pady=10, sticky="nsew",rowspan = 8)


    # Calculate
    def Calc_DE00(self,img_width,img_height):
        try: # obtain the entries (only numbers)
            L0 = float(self.entryL.get())
            a0 = float(self.entrya.get())
            b0 = float(self.entryb.get())
            dL = float(self.entrydL.get())
            da = float(self.entryda.get())
            db = float(self.entrydb.get())
        except:
            self.Result_label.configure(text="Only Numbers accepted")
        L = np.array([L0,dL])
        a = np.array([a0,da])
        b = np.array([b0,db])
        DE00 = deltaE2000(L,a,b,0)
        if DE00 < 2: # =crit
            DE00_color = "green" # aproved
        else:
            DE00_color = "red" # Rejected
        self.Result_label.configure(text= "%.2f" % DE00,text_color="white",fg_color=DE00_color) # Shows DE00 with 2 decimals

        # plot and show graphs
        graph(L,a,b,0,0)
        self.Chromatic_image = ctk.CTkImage(Image.open('Chromatic_plot2.png'),size=(img_width,img_height))
        self.Chromatic_image_label.configure(image=self.Chromatic_image)
        graphL(L,a,b,0,0)
        self.Light_image = ctk.CTkImage(Image.open('Light_plot2.png'),size=(img_width/2,img_height))
        self.Light_image_label.configure(image=self.Light_image)

    def Next(self): # Next measure
        self.entrydL.delete(0, ctk.END) # Clears the input text
        self.entryda.delete(0, ctk.END)
        self.entrydb.delete(0, ctk.END)
        #self.Chromatic_image_label.destroy
        #self.Light_image_label.destroy
        self.Chromatic_image_label.configure(image=self.Chromatic_image0)
        self.Light_image_label.configure(image=self.Light_image0)

    
    def Limpiar(self): # resets the display
        #self.Chromatic_image_label.img_ = self.Chromatic_image0 # blank image
        #self.Light_image_label.img_ = self.Light_image0
        self.Chromatic_image_label.configure(image=self.Chromatic_image0)
        self.Light_image_label.configure(image=self.Light_image0)
        self.entryL.delete(0, ctk.END)
        self.entrya.delete(0, ctk.END)
        self.entryb.delete(0, ctk.END)
        self.entrydL.delete(0, ctk.END)
        self.entryda.delete(0, ctk.END)
        self.entrydb.delete(0, ctk.END)
        self.entryRegister.delete(0, ctk.END)
        self.Result_label.configure(self,text="Resultado",fg_color=("gray50","gray50"))

    def Registrar(self):
        try:
            L0 = float(self.entryL.get())
            a0 = float(self.entrya.get())
            b0 = float(self.entryb.get())
        except:
            self.Result_label.configure(text="Only Numbers accepted")
        Entryname = self.entryRegister.get()
        Register(L0,a0,b0,Entryname)


#if __name__ == "__main__":
#    app = App()
#    app.mainloop()

In [ ]:
class App(ctk.CTk):
    
    def __init__(self):
        super().__init__()
        
        screen_width = self.winfo_screenwidth() # get current screen width and height
        screen_height = self.winfo_screenheight()
        img_width= screen_width/3 # Sets images width 
        img_height= screen_width/3 # Sets images height 
        
        self.title("Color Difference")
        self.geometry(f"{screen_width}x{screen_height}") # the f is to use variables inside ""
        # Imagen de fondo
        self.bg_image = ctk.CTkImage(Image.open('Fondo4.png'),size=(screen_width,screen_height))
        self.bg_image_label = ctk.CTkLabel(self, image=self.bg_image)
        self.bg_image_label.grid(row=0, column=0)

        ## Frames
        ### Starting frame
        self.starting_frame = ctk.CTkFrame(self,corner_radius=0)
        self.starting_frame.grid(row=0,column=0,sticky="ns")
        self.starting_label = ctk.CTkLabel(self.starting_frame,text="Página de inicio",
                                                  font=ctk.CTkFont(size=20, weight="bold"))
        self.starting_label.grid(row=0, column=0, padx=30, pady=(150, 15))
        self.main_frame_button = ctk.CTkButton(self.starting_frame,text="Main Frame",command=self.tomain_frame)
        self.main_frame_button.grid(row=3, column=0, padx=30, pady=(15, 15))
        self.register_frame_button = ctk.CTkButton(self.starting_frame,text="Register Frame",command=self.toregister_frame)
        self.register_frame_button.grid(row=4, column=0, padx=30, pady=(15, 15))

        ### main frame
        self.main_frame = ctk.CTkFrame(self,corner_radius=0)

        # set grid layout 
        self.main_frame.grid_columnconfigure(3, weight=5)
        self.main_frame.grid_columnconfigure(4, weight=3)

        # Load images       
        self.Chromatic_image0 = ctk.CTkImage(Image.open('Blank.png'),size=(img_width,img_height))
        self.Light_image0 = ctk.CTkImage(Image.open('Blank.png'),size=(img_width/2,img_height))

        # labels
        self.Title_label = ctk.CTkLabel(self.main_frame,text="Pruebas de Color",font=ctk.CTkFont(size=30, weight="bold"))
        self.Title_label.grid(row=0, column=0, pady=10,columnspan=5)

        self.Data_label = ctk.CTkLabel(self.main_frame,text="Datos",font=ctk.CTkFont(size=20))
        self.Data_label.grid(row=1, column=1, pady=10,columnspan=2)

        self.Data_label = ctk.CTkLabel(self.main_frame,text="Gráficas",font=ctk.CTkFont(size=20))
        self.Data_label.grid(row=1, column=3, pady=10,columnspan=2)

        self.Result_label = ctk.CTkLabel(self.main_frame,text="Resultado",font=ctk.CTkFont(size=20),fg_color=("gray50","gray50"),
                                         corner_radius=0)
        self.Result_label.grid(row=7, column=1, pady=10,sticky="nsew",columnspan=2)

        self.Patron_label = ctk.CTkLabel(self.main_frame,text="Patrón:",font=ctk.CTkFont(size=18))
        self.Patron_label.grid(row=2, column=1)#,padx=10)

        self.Diferencia_label = ctk.CTkLabel(self.main_frame,text="Diferencia:",font=ctk.CTkFont(size=18)) 
        self.Diferencia_label.grid(row=2, column=2)#,padx=10)

        self.L_label = ctk.CTkLabel(self.main_frame,text="L:") # ,font=ctk.CTkFont(size=20)
        self.L_label.grid(row=3, column=0)#,padx=10)

        self.a_label = ctk.CTkLabel(self.main_frame,text="a:") 
        self.a_label.grid(row=4, column=0)

        self.b_label = ctk.CTkLabel(self.main_frame,text="b:") 
        self.b_label.grid(row=5, column=0)

        self.DE_label = ctk.CTkLabel(self.main_frame,text="ΔE:") 
        self.DE_label.grid(row=7, column=0,padx=20)


        # entries
        entry_conf = {'corner_radius':5, 'fg_color':"blue",#("gray10","gray90"),# standard entry configuration
                    'text_color':("black")}
        
        self.entryL = ctk.CTkEntry(self.main_frame,placeholder_text="Inserte un número",**entry_conf)
        self.entryL.grid(row=3, column=1)

        self.entrya = ctk.CTkEntry(self.main_frame,placeholder_text="Inserte un número",**entry_conf)
        self.entrya.grid(row=4, column=1)

        self.entryb = ctk.CTkEntry(self.main_frame,placeholder_text="Inserte un número",**entry_conf)
        self.entryb.grid(row=5, column=1)

        self.entrydL = ctk.CTkEntry(self.main_frame,placeholder_text="Inserte la diferencia",**entry_conf)
        self.entrydL.grid(row=3, column=2)

        self.entryda = ctk.CTkEntry(self.main_frame,placeholder_text="Inserte la diferencia",**entry_conf)
        self.entryda.grid(row=4, column=2)

        self.entrydb = ctk.CTkEntry(self.main_frame,placeholder_text="Inserte la diferencia",**entry_conf)
        self.entrydb.grid(row=5, column=2)

        # Buttons
        # "lambda:" allows to call the function with arguments (normally it isn't possible for bottons)
        self.calcular_button = ctk.CTkButton(self.main_frame,corner_radius=4,text="Calcular",fg_color=("#1f6aa5"),
                                          command=lambda:self.Calc_DE00(img_width,img_height),font=ctk.CTkFont(size=20))
        self.calcular_button.grid(row=6, column=1, pady=10, sticky="ns",columnspan=2)

        self.siguiente_button = ctk.CTkButton(self.main_frame,corner_radius=4,text="Siguiente Medida",fg_color=("#1f6aa5"),
                                          command=self.Next,font=ctk.CTkFont(size=20))
        self.siguiente_button.grid(row=8, column=1, pady=10, sticky="ns",columnspan=2)
        
        self.Limpiar_button = ctk.CTkButton(self.main_frame,corner_radius=4,text="Limpiar",fg_color=("#1f6aa5"),
                                          command=self.Limpiar,font=ctk.CTkFont(size=20))
        self.Limpiar_button.grid(row=10, column=3, pady=10, sticky="ns")

        self.register_frame_button2 = ctk.CTkButton(self.main_frame,corner_radius=4,text="Registrar",fg_color=("#1f6aa5"),
                                          command=self.toregister_frame,font=ctk.CTkFont(size=20))
        self.register_frame_button2.grid(row=10, column=4, pady=10, sticky="ns")

        # Images
        self.Chromatic_image_label = ctk.CTkLabel(self.main_frame,text="",image=self.Chromatic_image0,compound="left")
        self.Chromatic_image_label.grid(row=2, column=3, pady=10, sticky="nsew",rowspan = 8)

        self.Light_image_label = ctk.CTkLabel(self.main_frame,text="",image=self.Light_image0,compound="left")
        self.Light_image_label.grid(row=2, column=4, pady=10, sticky="nsew",rowspan = 8)


        ### Register frame
        self.register_frame = ctk.CTkFrame(self,corner_radius=0)

        self.register_Title_label = ctk.CTkLabel(self.register_frame,text="Registro",font=ctk.CTkFont(size=40, weight="bold"))
        self.register_Title_label.grid(row=0, column=0, pady=10,columnspan=10)

        self.registrar_button = ctk.CTkButton(self.register_frame,corner_radius=4,text="Guardar",fg_color=("#1f6aa5"),
                                          command=self.Registrar,font=ctk.CTkFont(size=20))
        self.registrar_button.grid(row=10, column=3, pady=10, sticky="ns")

        self.Subtitle_label = ctk.CTkLabel(self.register_frame,text="Datos de la muestra patrón:",font=ctk.CTkFont(size=20)) # ,font=ctk.CTkFont(size=20)
        self.Subtitle_label.grid(row=2, column=1,padx=10,pady=10)

        self.L_label = ctk.CTkLabel(self.register_frame,text="L:") # ,font=ctk.CTkFont(size=20)
        self.L_label.grid(row=3, column=0,padx=10)

        self.a_label = ctk.CTkLabel(self.register_frame,text="a:") 
        self.a_label.grid(row=4, column=0)

        self.b_label = ctk.CTkLabel(self.register_frame,text="b:") 
        self.b_label.grid(row=5, column=0)

        self.register_label = ctk.CTkLabel(self.register_frame,text="Nombre del registro:") 
        self.register_label.grid(row=6, column=0,padx=20)

        entry_conf2 = {'corner_radius':5, 'fg_color':("gray10","gray90"),# standard entry configuration
                    'text_color':("black")}
       
        self.entryL = ctk.CTkEntry(self.register_frame,placeholder_text="Inserte un número",**entry_conf2)
        self.entryL.grid(row=3, column=1)

        self.entrya = ctk.CTkEntry(self.register_frame,placeholder_text="Inserte un número",**entry_conf2)
        self.entrya.grid(row=4, column=1)

        self.entryb = ctk.CTkEntry(self.register_frame,placeholder_text="Inserte un número",**entry_conf2)
        self.entryb.grid(row=5, column=1)

        self.entryRegister = ctk.CTkEntry(self.register_frame,placeholder_text="Escriba el nombre",**entry_conf2)
        self.entryRegister.grid(row=6, column=1)#"""

        ### Error frame
        self.error_frame = ctk.CTkFrame(self,corner_radius=0)
        self.error_frame.grid(row=0,column=0)

        self.error_label = ctk.CTkLabel(self.error_frame,text="Para registrar una muestra se necesitan todos lo valores y un nombre") 
        self.error_label.grid(row=1, column=1,padx=20)

        self.error_close_button = ctk.CTkButton(self.error_label,corner_radius=3,text="Cerrar",fg_color=("#F81134"),
                                          command=self.cerrar_error,font=ctk.CTkFont(size=20))
        self.registrar_button.grid(row=3, column=3, pady=10, sticky="ns")


    ##### Functions
    def tomain_frame(self):
        self.starting_frame.grid_forget()
        self.main_frame.grid(row=0, column=0, sticky="nsew", padx=100)

    def toregister_frame(self):
        self.starting_frame.grid_forget()
        self.register_frame.grid(row=0, column=0, sticky="nsew", padx=100)

    def toerror_frame(self):
        self.error_frame.grid(row=1, column=1,padx=100)  
    
    def cerrar_error(self):
        self.error_frame.grid_forget()
    
    # Calculate
    def Calc_DE00(self,img_width,img_height):
        try: # obtain the entries (only numbers)
            L0 = float(self.entryL.get())
            a0 = float(self.entrya.get())
            b0 = float(self.entryb.get())
            dL = float(self.entrydL.get())
            da = float(self.entryda.get())
            db = float(self.entrydb.get())
        except:
            self.Result_label.configure(text="Only Numbers accepted")
        L = np.array([L0,dL])
        a = np.array([a0,da])
        b = np.array([b0,db])
        DE00 = deltaE2000(L,a,b,0)
        if DE00 < 2: # =crit
            DE00_color = "green" # aproved
        else:
            DE00_color = "red" # Rejected
        self.Result_label.configure(text= "%.2f" % DE00,text_color="white",fg_color=DE00_color) # Shows DE00 with 2 decimals

        # plot and show graphs
        graph(L,a,b,0,0)
        self.Chromatic_image = ctk.CTkImage(Image.open('Chromatic_plot2.png'),size=(img_width,img_height))
        self.Chromatic_image_label.configure(image=self.Chromatic_image)
        graphL(L,a,b,0,0)
        self.Light_image = ctk.CTkImage(Image.open('Light_plot2.png'),size=(img_width/2,img_height))
        self.Light_image_label.configure(image=self.Light_image)

    def Next(self): # Next measure
        self.entrydL.delete(0, ctk.END) # Clears the input text
        self.entryda.delete(0, ctk.END)
        self.entrydb.delete(0, ctk.END)
        #self.Chromatic_image_label.destroy
        #self.Light_image_label.destroy
        self.Chromatic_image_label.configure(image=self.Chromatic_image0)
        self.Light_image_label.configure(image=self.Light_image0)

    
    def Limpiar(self): # resets the display
        #self.Chromatic_image_label.img_ = self.Chromatic_image0 # blank image
        #self.Light_image_label.img_ = self.Light_image0
        self.Chromatic_image_label.configure(image=self.Chromatic_image0)
        self.Light_image_label.configure(image=self.Light_image0)
        self.entryL.delete(0, ctk.END)
        self.entrya.delete(0, ctk.END)
        self.entryb.delete(0, ctk.END)
        self.entrydL.delete(0, ctk.END)
        self.entryda.delete(0, ctk.END)
        self.entrydb.delete(0, ctk.END)
        self.entryRegister.delete(0, ctk.END)
        self.Result_label.configure(self,text="Resultado",fg_color=("gray50","gray50"))

    def Registrar(self):
        try:
            L0 = float(self.entryL.get())
            a0 = float(self.entrya.get())
            b0 = float(self.entryb.get())
        except:
            self.Result_label.configure(text="Only Numbers accepted")
            toerror_frame() # WTF
        Entryname = self.entryRegister.get()
        Register(L0,a0,b0,Entryname)

        

        


if __name__ == "__main__":
    app = App()
    app.mainloop()